In [1]:
%%writefile mapper-tf.py
from __future__ import print_function
import sys
import re
import string

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'
tagWord = 'wordN'
tagWords = 'totalWordsNt'

stopWords = set()

with open(path) as stopWordsFile:
    for line in stopWordsFile:
        try:         
            stopWords.add(unicode(line.strip().lower()))
        except ValueError as e:
            continue

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        print("reporter:counter:Wiki stats,Wrong articles found,%d" % 1, file=sys.stderr)
        continue

    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    wordsInArticle = 0
    
    for word in words:
        word = word.strip(string.punctuation).lower()
        if word == "":
            print("reporter:counter:Wiki stats,Null words found,%d" % 1, file=sys.stderr)
            continue
        if word in stopWords:
            print("reporter:counter:Wiki stats,Stop words found,%d" % 1, file=sys.stderr)
            continue
        wordsInArticle += 1
        print("reporter:counter:Wiki stats,Total words found,%d" % 1, file=sys.stderr)
        print(article_id, tagWord, word, 1, sep='\t')

    print(article_id, tagWords, "-", wordsInArticle, sep='\t')


Writing mapper-tf.py


In [79]:
%%writefile reducer-tf.py
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_article_id = None
current_word = None
word_sum, wordsInArticle = 0, 0

tagWord = 'wordN'
tagWords = 'totalWordsNt'
tagTf = 'tf'

for line in sys.stdin:
    try:
        article_id, tag, word, count = unicode(line.strip()).split('\t')
        #print "after parse:", article_id, word, count
        if article_id == "" or tag == "" or word == "" or count == "":   # sanity check
            continue
        count = int(count)
    except ValueError as e:
        #print "Parse error:", e
        continue

    if current_article_id != article_id:
        if tag == tagWords: # get total words in article
            wordsInArticle = count
            continue
        if current_article_id != None:
            print "%s\t%s\t%s\t%f" % (current_word, tagTf, current_article_id, 1.0*word_sum/wordsInArticle)
            word_sum = 0
            current_word = word
        current_article_id = article_id
        #print "wordsInArticle:", word, wordsInArticle
        
    if current_word != word:
        if current_word != None:
            print "%s\t%s\t%s\t%f" % (current_word, tagTf, current_article_id, 1.0*word_sum/wordsInArticle)
            #print "%s\t%s\t%d\t%d" % (current_article_id, current_word, word_sum, wordsInArticle)
        word_sum = 0
        current_word = word
    word_sum += count

if current_article_id:
    print "%s\t%s\t%s\t%f" % (current_word, tagTf, current_article_id, 1.0*word_sum/wordsInArticle)


Overwriting reducer-tf.py


In [84]:
%%writefile mapper-idf.py
from __future__ import print_function
import sys

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

for line in sys.stdin:
    word, _, _, _ = unicode(line.strip()).split('\t')
    print("reporter:counter:Idf stats,Total words found,%d" % 1, file=sys.stderr)
    print(word, 1, sep='\t')


Writing mapper-idf.py


In [85]:
%%writefile reducer-idf.py
import sys
from math import log

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_key = None
word_sum = 0
tag = "idf"

for line in sys.stdin:
    try:
        key, count = unicode(line.strip()).split('\t', 1)
        if key == "" or count == "":    # sanity check
            continue
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%s\t%f" % (current_key, tag, (1.0/log(1.0 + word_sum)))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%s\t%f" % (current_key, tag, (1.0/log(1.0 + word_sum)))

Writing reducer-idf.py


In [92]:
%%writefile mapper-tf-idf.py
from __future__ import print_function
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

tf = "tf"
idf = "idf"

for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    if len(inVal) > 4:
        print("reporter:counter:Wiki stats,tf idf > 4 parsing errors,%d" % 1, file=sys.stderr)
        continue
    try:
        if inVal[1] == idf: # idf data
            print("reporter:counter:Wiki stats,idf words,%d" % 1, file=sys.stderr)
            print(inVal[0], idf, float(inVal[2]), sep='\t')
            
        elif inVal[1] == tf: # tf data
            print("reporter:counter:Wiki stats,tf words,%d" % 1, file=sys.stderr)
            print(inVal[0], tf, inVal[2], float(inVal[3]), sep='\t')
            
        else:
            print("reporter:counter:Wiki stats,tf idf last else parsing errors,%d" % 1, file=sys.stderr)
    except ValueError as e:
        print("reporter:counter:Wiki stats,tf idf to float errors,%d" % 1, file=sys.stderr)
        continue


Writing mapper-tf-idf.py


In [93]:
%%writefile reducer-tf-idf.py
import sys
from math import log

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

tf = "tf"
idf = "idf"

for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    if (len(inVal) < 2 and len(inVal) > 4):      # sanity check
        continue
    try:
        if inVal[1] == idf:
            idfWord, _, idfDt = inVal
            idfDt = float(idfDt)

        if inVal[1] == tf:
            tfWord, _, article_id, tfNt = inVal
            tfNt = float(tfNt)
            if idfWord == tfWord:
                print "%s\t%s\t%f" % (idfWord, article_id, tfNt*idfDt)

    except ValueError as e:
        continue


Writing reducer-tf-idf.py


In [8]:
!hdfs dfs -mkdir data

In [9]:
!hdfs dfs -put wiki-test.txt data

In [36]:
!ls -l /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar

lrwxrwxrwx 1 root root 61 Oct 17  2017 /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar -> /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.8.1.jar


In [80]:
%%bash

OUT_DIR_TF="tf_result"
#OUT_DIR="/data/wiki/test/tf-out"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D stream.num.map.output.key.fields=3 \
-D stream.num.reduce.output.fields=3 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapreduce.partition.keycomparator.options=-k1,3 \
-D mapreduce.job.name="Tf-Idf tf task" \
-D mapreduce.job.reduces=${NUM_REDUCERS} \
-files mapper-tf.py,reducer-tf.py,/datasets/stop_words_en.txt  \
-mapper "python mapper-tf.py" \
-reducer "python reducer-tf.py" \
-input /data/wiki/en_articles_part \
-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-output ${OUT_DIR_TF} 

packageJobJar: [/tmp/hadoop-unjar981246973952476236/] [] /tmp/streamjob1345325328332772262.jar tmpDir=null


19/02/08 07:13:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 07:13:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 07:13:50 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/08 07:13:50 INFO mapreduce.JobSubmitter: number of splits:2
19/02/08 07:13:50 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549545179555_0023
19/02/08 07:13:50 INFO impl.YarnClientImpl: Submitted application application_1549545179555_0023
19/02/08 07:13:50 INFO mapreduce.Job: The url to track the job: http://e5801941607d:8088/proxy/application_1549545179555_0023/
19/02/08 07:13:50 INFO mapreduce.Job: Running job: job_1549545179555_0023
19/02/08 07:13:56 INFO mapreduce.Job: Job job_1549545179555_0023 running in uber mode : false
19/02/08 07:13:56 INFO mapreduce.Job:  map 0% reduce 0%
19/02/08 07:14:13 INFO mapreduce.Job:  map 15% reduce 0%
19/02/08 07:14:19 INFO mapreduce.Job:  map 25% reduce 0%
19/02/08 07:14:25 INFO 

In [81]:
! hdfs dfs -cat tf_result/* | head -100

0-396-08747-7	tf	1002	0.000578
1	tf	1002	0.003470
11	tf	1002	0.000578
12	tf	1002	0.000578
13	tf	1002	0.000578
14	tf	1002	0.000578
15	tf	1002	0.000578
16	tf	1002	0.000578
1863	tf	1002	0.000578
1869	tf	1002	0.000578
1926	tf	1002	0.001157
1927	tf	1002	0.000578
1930	tf	1002	0.001735
1932	tf	1002	0.001157
1939	tf	1002	0.000578
1940	tf	1002	0.001157
1940s	tf	1002	0.000578
1942	tf	1002	0.000578
1943	tf	1002	0.000578
1950	tf	1002	0.001735
1952	tf	1002	0.001735
1953	tf	1002	0.001157
1954	tf	1002	0.000578
1956	tf	1002	0.000578
1957	tf	1002	0.001735
1960	tf	1002	0.000578
1960s	tf	1002	0.000578
1961	tf	1002	0.000578
1962	tf	1002	0.001157
1963	tf	1002	0.000578
1964	tf	1002	0.002313
1965	tf	1002	0.000578
1965).the	tf	1002	0.000578
1970	tf	1002	0.000578
1971	tf	1002	0.000578
1974	tf	1002	0.000578
1976	tf	1002	0.001157
1977	tf	1002	0.000578
1979	tf	1002	0.000578
1980	tf	1002	0.000578
1983	tf	1002	0.001157
1984	tf	1002	0.001735
1985	tf	1002	0.002313
1986	tf	1002	0.000578
1987	tf	1002	0.002313
1989	tf	1

In [82]:
! hdfs dfs -getmerge tf_result/* tf.csv

In [83]:
!tail tf.csv

worsen	tf	986	0.000361
worsening	tf	986	0.000361
write	tf	986	0.000361
writing	tf	986	0.000722
written	tf	986	0.000361
year	tf	986	0.000361
years	tf	986	0.000722
yield	tf	986	0.000361
young	tf	986	0.001445
zest	tf	986	0.000361


In [86]:
%%bash
#OUT_DIR_IDF="idf_result_"$(date +"%s%6N")
OUT_DIR_TF="tf_result"
OUT_DIR_IDF="idf_result"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Tf-Idf idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-idf.py,reducer-idf.py \
    -mapper "python mapper-idf.py" \
    -reducer "python reducer-idf.py" \
    -input ${OUT_DIR_TF} \
    -output ${OUT_DIR_IDF} > /dev/null



rm: `idf_result': No such file or directory
19/02/08 07:20:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 07:20:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 07:20:23 INFO mapred.FileInputFormat: Total input files to process : 3
19/02/08 07:20:24 INFO mapreduce.JobSubmitter: number of splits:3
19/02/08 07:20:24 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549545179555_0024
19/02/08 07:20:25 INFO impl.YarnClientImpl: Submitted application application_1549545179555_0024
19/02/08 07:20:25 INFO mapreduce.Job: The url to track the job: http://e5801941607d:8088/proxy/application_1549545179555_0024/
19/02/08 07:20:25 INFO mapreduce.Job: Running job: job_1549545179555_0024
19/02/08 07:20:32 INFO mapreduce.Job: Job job_1549545179555_0024 running in uber mode : false
19/02/08 07:20:32 INFO mapreduce.Job:  map 0% reduce 0%
19/02/08 07:20:50 INFO mapreduce.Job:  map 89% reduce 0%
19/02/08 07:20:51 INFO mapreduce.Job

In [87]:
! hdfs dfs -ls idf_result

Found 4 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-08 07:21 idf_result/_SUCCESS
-rw-r--r--   1 jovyan supergroup    3326152 2019-02-08 07:20 idf_result/part-00000
-rw-r--r--   1 jovyan supergroup    3338790 2019-02-08 07:21 idf_result/part-00001
-rw-r--r--   1 jovyan supergroup    3325521 2019-02-08 07:21 idf_result/part-00002


In [88]:
! hdfs dfs -getmerge idf_result/* idf.csv

In [89]:
! head idf.csv

0%however	idf	1.442695
0&\mathrm{if	idf	1.442695
0&\mbox{if	idf	1.442695
0)).(1	idf	1.442695
0).in	idf	1.442695
0).one	idf	1.442695
0+1	idf	1.442695
0,0	idf	0.455120
0,00	idf	1.442695
0,03	idf	1.442695


In [90]:
! tail idf.csv

𐑈	idf	1.442695
𐑋	idf	1.442695
𐑎	idf	1.442695
𐡁	idf	1.442695
𐡄	idf	1.442695
𐡇	idf	1.442695
𐡊	idf	1.442695
𐡍	idf	1.442695
𐡐	idf	1.442695
𐡓	idf	1.442695


In [94]:
%%bash
OUT_DIR_TF="tf_result"
OUT_DIR_IDF="idf_result"
#OUT_DIR_TF_IDF="tf_idf_result_"$(date +"%s%6N")
OUT_DIR_TF_IDF="tf-idf-out"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D stream.num.map.output.key.fields=2 \
-D stream.num.reduce.output.fields=2 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapreduce.partition.keycomparator.options=-k1,2 \
-D mapreduce.job.name="Tf-Idf tf*idf task" \
-D mapreduce.job.reduces=${NUM_REDUCERS} \
-files mapper-tf-idf.py,reducer-tf-idf.py \
-mapper "python mapper-tf-idf.py" \
-reducer "python reducer-tf-idf.py" \
-input ${OUT_DIR_TF},${OUT_DIR_IDF} \
-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-output ${OUT_DIR_TF_IDF} > /dev/null



rm: `tf-idf-out': No such file or directory
19/02/08 07:35:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 07:35:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 07:35:52 INFO mapred.FileInputFormat: Total input files to process : 6
19/02/08 07:35:52 INFO mapreduce.JobSubmitter: number of splits:6
19/02/08 07:35:53 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549545179555_0025
19/02/08 07:35:53 INFO impl.YarnClientImpl: Submitted application application_1549545179555_0025
19/02/08 07:35:53 INFO mapreduce.Job: The url to track the job: http://e5801941607d:8088/proxy/application_1549545179555_0025/
19/02/08 07:35:53 INFO mapreduce.Job: Running job: job_1549545179555_0025
19/02/08 07:35:59 INFO mapreduce.Job: Job job_1549545179555_0025 running in uber mode : false
19/02/08 07:35:59 INFO mapreduce.Job:  map 0% reduce 0%
19/02/08 07:36:28 INFO mapreduce.Job: Task Id : attempt_1549545179555_0025_m_000002_0, Status

In [95]:
!hdfs dfs -ls tf-idf-out

Found 4 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-08 07:37 tf-idf-out/_SUCCESS
-rw-r--r--   1 jovyan supergroup   25513964 2019-02-08 07:37 tf-idf-out/part-00000
-rw-r--r--   1 jovyan supergroup   26007862 2019-02-08 07:37 tf-idf-out/part-00001
-rw-r--r--   1 jovyan supergroup   25228283 2019-02-08 07:37 tf-idf-out/part-00002


In [98]:
! hdfs dfs -getmerge tf-idf-out/* tf-idf.csv

In [99]:
!wc tf-idf.csv

 3472203 10416609 76750109 tf-idf.csv


In [105]:
!grep -E 'labor\W12\W' tf-idf.csv

labor	12	0.000351


In [106]:
!hdfs dfs -text tf-idf-out/* | grep -E 'labor\W12\W'

labor	12	0.000351
